In [1]:
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import multivariate_normal

In [21]:
# TODO suturation action, reward state

In [36]:
class DNetwork(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(DNetwork, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        hiden_layer_len_1 = 50
#         hiden_layer_len_2 = 50
        self.hidden_1 = nn.Linear(self.in_dim, hiden_layer_len_1, bias=True)
#         self.hidden_2 = nn.Linear(hiden_layer_len_1, hiden_layer_len_2, bias=True)
        self.output = nn.Linear(hiden_layer_len_1, self.out_dim, bias=True)
        self.activation_on_hidden_1 =  nn.ReLU() # nn.Tanh() # nn.ReLU()
#         self.activation_on_hidden_2 = nn.ReLU() #
        self.activation_on_output = nn.ReLU() # nn.Tanh()  # nn.Softmax()

    def forward(self, x):
        print(x)
        x = self.hidden_1(x)
        print(x)
        x = self.activation_on_hidden_1(x)
        print(x)
#         x = self.hidden_2(x)
#         x = self.activation_on_hidden_2(x)
        x = self.output(x)
        print(x)
        x = self.activation_on_output(x)
        print(x)
        # x = self.Norm(x, self.sigma*torch.eye(2)).sample()
        # torch.maximum(torch.minimum(x, torch.tensor(1)), torch.tensor(-1)) # self.activation_on_output(x) #
        return x    

In [46]:
class Dynamics:
    def __init__(self, state_dim=3, action_dim=1):
        self.model = DNetwork(in_dim=(state_dim+action_dim), out_dim=state_dim)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.model.parameters(), lr=0.002)
    
    def prepare_input(self, state: list, action: float) -> np.ndarray:
        s = np.array(state)
#         s = s.reshape((s.shape[0], 1))
        a = np.array([action])
#         a = a.reshape((a.shape[0], 1))
        return np.concatenate((s, a), axis=0)
    
    def step(self, state: list, action: float) -> list:
        s_a = self.prepare_input(state, action)
        with torch.no_grad():
            state_next = self.model(torch.from_numpy(s_a).float()) # .detach().numpy()
        # list(map(lambda x: x[0], state_next.detach().numpy()))
        state_next = list(state_next.detach().numpy())
        return state_next
    
    def fit_one_object(self, states_actions: np.ndarray, next_states: np.ndarray):
        states_actions = torch.from_numpy(states_actions).float()
        next_states = torch.from_numpy(next_states).float()
        self.optimizer.zero_grad()
        guessies = self.model(states_actions)
        loss = self.criterion(guessies, next_states)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [47]:
dyn = Dynamics()

In [48]:
print(episode_trajectory_init_state)
print(episode_trajectory_first_action)

[-0.4491204619407654, 0.26583366334342057, 0.0]
2


In [49]:
next_state = dyn.step(episode_trajectory_init_state, episode_trajectory_first_action)

tensor([-0.4491,  0.2658,  0.0000,  2.0000])
tensor([-1.2929, -0.8188,  1.5273, -0.2378,  0.2018, -0.9289, -0.9905, -0.4895,
        -0.0903, -0.2861,  0.4248,  0.8645, -0.8494,  1.3551, -0.4520,  0.8783,
         0.7722, -0.1235, -0.5800, -0.2291,  0.2927, -0.6128, -0.7505, -0.0428,
         0.3597, -0.8008, -0.2097,  0.5119, -0.4588, -0.4743, -0.1361, -0.2432,
         0.1919,  0.7825, -0.9384,  0.1284,  0.2247, -0.3597,  1.0984,  0.9165,
        -0.9597,  0.2246, -0.0681, -0.4799, -0.5841, -0.5992, -0.4357, -0.1403,
         0.8285,  0.1677])
tensor([0.0000, 0.0000, 1.5273, 0.0000, 0.2018, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.4248, 0.8645, 0.0000, 1.3551, 0.0000, 0.8783, 0.7722, 0.0000,
        0.0000, 0.0000, 0.2927, 0.0000, 0.0000, 0.0000, 0.3597, 0.0000, 0.0000,
        0.5119, 0.0000, 0.0000, 0.0000, 0.0000, 0.1919, 0.7825, 0.0000, 0.1284,
        0.2247, 0.0000, 1.0984, 0.9165, 0.0000, 0.2246, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.8285, 0.1677])

In [4]:
class Policy:
    def __init__(self, state_dim=3, action_dim=1):
        self.model = DNetwork(in_dim=state_dim, out_dim=action_dim)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.model.parameters(), lr=0.002)
        
    def get_action(self, state: list) -> float:
        s = prepare_input(state)
        with torch.no_grad():
            action = self.model(torch.from_numpy(s).float()) # .detach().numpy()
        action = action.detach().numpy()[0]
        return action
    
    def fit_one_object(self, states: np.ndarray, actions: np.ndarray):
        states = torch.from_numpy(states).float()
        actions = torch.from_numpy(actions).float()
        self.optimizer.zero_grad()
        guessies = self.model(states)
        loss = self.criterion(guessies, actions)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [5]:
class Reward:
    def __init__(self, state_dim=3, action_dim=1):
        self.model = DNetwork(in_dim=(state_dim+action_dim), out_dim=1)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.model.parameters(), lr=0.002)
    
    def prepare_input(state: list, action: float) -> np.ndarray:
        s = np.array(state)
        s = s.reshape((s.shape[0], 1))
        a = np.array([action])
        a = a.reshape((a.shape[0], 1))
        s_a = np.concatenate((s, a), axis=0)
    
    def calculate(self, state: list, action: float) -> float:
        s_a = prepare_input(state, action)
        with torch.no_grad():
            reward = self.model(torch.from_numpy(s_a).float()) # .detach().numpy()
        reward = reward.detach().numpy()[0]
        return reward
    
    def fit_one_object(self, states_actions: np.ndarray, rewards: np.ndarray):
        states_actions = torch.from_numpy(states_actions).float()
        rewards = torch.from_numpy(rewards).float()
        self.optimizer.zero_grad()
        guessies = self.model(states_actions)
        loss = self.criterion(guessies, rewards)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [6]:
policy = Policy()
reward_function = Reward()
dynamics = Dynamics()

In [7]:
def V(init_state, horizon):
    gamma = 0.9
    value = 0
    state = init_state
    for t in range(1, horizon):
        action = policy.get_action(state)
        reward = reward_function.calculate(state, action)
        value += (gamma**t) * reward
        state = dynamics.step(state, action)
    return value

In [8]:
def E_of_V(init_state, horizon): # TODO horizon не дан
    N = 100
    V_avg = sum([V(init_state, horizon) for _ in range(N)]) / N

In [9]:
# prepare data

In [10]:
import json

In [11]:
with open('train.json', 'r') as json_file:
    train = json.load(json_file)

In [17]:
episode = 4

episode_trajectory = train[episode]

print(len(episode_trajectory["states"]))
print(len(episode_trajectory["actions"]))
print(len(episode_trajectory["rewards"]))
print(episode_trajectory["truncated"])


344
343
343


False

In [19]:
episode_trajectory_init_state = episode_trajectory["states"][0]

In [20]:
episode_trajectory_init_state

[-0.4491204619407654, 0.26583366334342057, 0.0]

In [23]:
episode_trajectory_first_action = episode_trajectory["actions"][0]

In [24]:
episode_trajectory_first_action

2

In [ ]:
# fit models

In [ ]:
# submit solution

In [ ]:
import json
from random import random


def main():
    # считываем данные
    with open('train.json', 'r') as json_file:
        train = json.load(json_file)

    # посмотрим на информацию, доступную в нулевом эпизоде
    print(train[0].keys())

    # напечатаем возможные действия
    print(set(train[0]['actions']))

    # считываем данные, для которых нужно сделать предсказание
    with open('submit.json', 'r') as json_file:
        submit = json.load(json_file)

    # заполним поле V случайными числами
    for idx, item in enumerate(submit):
        state = item['state']
        item['V'] = random()

    # записываем предсказания в файл
    with open('solution.json', 'w') as json_file:
        json.dump(submit, json_file)


In [ ]:

class Agent:

    def __init__(self):
        self.model = DNetwork()

        self.model_shapes = []

        orig_model = copy.deepcopy(self.model)

        for param in orig_model.parameters():
            p = param.data.cpu().numpy()
            self.model_shapes.append(p.shape)

        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.model.parameters(), lr=0.002)


#         self.Norm = torch.distributions.multivariate_normal.MultivariateNormal

#         self.sigma = torch.tensor(0.1)

        self.sigma = 0.001

        self.last_laier = []


    def updateParams(self, flat_param: np.array):
        idx = 0
        i = 0

        for param in self.model.parameters():
            delta = np.product(self.model_shapes[i])
            block = flat_param[idx:idx + delta]
            block = np.reshape(block, self.model_shapes[i])
            i += 1
            idx += delta
            block_data = torch.from_numpy(block).float()

            param.data = block_data


    def getAction(self, state):

        with torch.no_grad():

            mu = self.model(torch.from_numpy(state).float()) # .detach().numpy()

        self.last_laier = copy.deepcopy(mu.numpy())

        action = multivariate_normal(mean=mu.numpy(), cov=self.sigma*np.eye(self.model.action_dim)).rvs(size=1) # copy.deepcopy(mu.numpy()) #

        np.clip(action, -1, 1, out= action)

        return action

    def fit(self, states, actions):

        states = torch.from_numpy(states).float()
        actions = torch.from_numpy(actions).float()

        #self.model.train()

        self.optimizer.zero_grad()

        guessies = self.model(states)
        loss = self.criterion(guessies, actions)
        loss.backward()
        self.optimizer.step()

        return loss.item()